**Recuerde no agregar o quitar celdas en este notebook, ni modificar su tipo. Si lo hace, el sistema automaticamente lo calificará con cero punto cero (0.0)**

Para resolver los problemas presentados, use el siguiente conjunto de datos. Use [gradetool](gradetool.md) para verificar las respuestas del notebook.

In [1]:
%load_ext bigdata
%hive_init

/home/jose/anaconda3.1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/jose/anaconda3.1/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Hive initialized!


In [3]:
%%writefile tbl0.csv
1,D,4,2016-06-25,a:e:c:d,bb#10:dd#20:cc#40
2,C,4,2015-05-14,a:c,dd#40:bb#20:cc#10
3,D,6,2014-12-26,b:d,aa#10:bb#40
4,D,5,2016-06-25,a:c:e:b:d,bb#40:dd#20:aa#10:cc#30
5,C,7,2016-05-23,d:e:c,cc#10:aa#20
6,A,2,2018-06-14,a:d,aa#20
7,B,3,2014-12-22,a:e:d,cc#20
8,C,6,2015-08-20,d:a:c:e,cc#10:aa#20
9,B,3,2017-12-08,b:a:c:e,cc#40:dd#10:aa#30:bb#20
10,B,7,2015-07-28,c:d:e:a:b,aa#10:dd#40:cc#30

Overwriting tbl0.csv


In [2]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

--SELECT * FROM tbl0;

In [4]:
%%writefile tbl1.csv
1,7,bb,A#0:B#4:C#1:D#3:E#5
2,13,Cc,A#4:B#1:C#0:D#5:E#2
3,11,dD,A#5:B#4:C#3:D#1:E#0
4,3,BB,A#2:B#4:C#3:D#5:E#1
5,14,cc,A#4:B#0:C#2:D#5:E#3
6,8,cC,A#4:B#5:C#1:D#2:E#3
7,2,DD,A#0:B#5:C#2:D#4:E#3
8,6,dd,A#4:B#2:C#5:D#3:E#0
9,9,aa,A#1:B#4:C#2:D#5:E#3
10,4,Bb,A#2:B#3:C#4:D#1:E#0

Overwriting tbl1.csv


In [ ]:
%%hive
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';

LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

SELECT * FROM tbl1;

# Problema 1

Escriba una consulta que retorne los valores únicos de la columna `t0.c5` (ordenados). 

In [10]:
%%hive
SELECT DISTINCT(value) FROM tbl0
lateral view explode(c5) dummy_table as value;

a
b
c
d
e


# Problema 2

Realice una consulta que compute la cantidad de veces que aparece cada valor de la columna `t0.c5`  por año.

In [9]:
%%hive
SELECT DATE_FORMAT(c4, "yyyy") as fecha, value as val, COUNT(value) FROM tbl0
lateral view explode(c5) dummy_table as value
GROUP BY value, DATE_FORMAT(c4, "yyyy")
ORDER BY fecha, val;  

2014	a	1
2014	b	1
2014	d	2
2014	e	1
2015	a	3
2015	b	1
2015	c	3
2015	d	2
2015	e	2
2016	a	2
2016	b	1
2016	c	3
2016	d	3
2016	e	3
2017	a	1
2017	b	1
2017	c	1
2017	e	1
2018	a	1
2018	d	1


# Problema 3

Escriba una consulta que retorne los elementos de la columna t0.c5 en mayúsculas, tal como se muestra a continuación.

Original:

    ["a","e","c","d"]
    ["a","c"]
    ["b","d"]
    ["a","c","e","b","d"]
    ["d","e","c"]
    ["a","d"]
    ["a","e","d"]
    ["d","a","c","e"]
    ["b","a","c","e"]
    ["c","d","e","a","b"]

Transformada:

    ["A","E","C","D"]
    ["A","C"]
    ["B","D"]
    ["A","C","E","B","D"]
    ["D","E","C"]
    ["A","D"]
    ["A","E","D"]
    ["D","A","C","E"]
    ["B","A","C","E"]
    ["C","D","E","A","B"]

In [6]:
%%hive
SELECT collect_set(UPPER(value)) 
FROM tbl0 LATERAL VIEW explode(c5) dummy_table as value
GROUP BY c1;

["A","E","C","D"]
["A","C"]
["B","D"]
["A","C","E","B","D"]
["D","E","C"]
["A","D"]
["A","E","D"]
["D","A","C","E"]
["B","A","C","E"]
["C","D","E","A","B"]


# Problema 4

Escriba una consulta que retorne para cada valor único de la columna `t0.c2`, los valores correspondientes de la columna `
t0.c1`. 

In [7]:
%%hive
SELECT c2, collect_set(c1)
FROM tbl0
GROUP BY c2;

A	[6]
B	[7,9,10]
C	[2,5,8]
D	[1,3,4]


# Problema 5

Escriba una consulta que para cada valor único de la columna `t0.c2,` calcule la suma de todos los valores asociados a las claves en la columna `t0.c6`.

In [6]:
%%hive

SELECT c2, SUM(value)
FROM tbl0
lateral view explode(c6) dummy_table AS key, value
GROUP BY c2;

A	20
B	200
C	130
D	220


# Problema 6

Escriba una consulta que retorne la columna `tbl0.c1` y el valor correspondiente de la columna `tbl1.c4` para la columna `tbl0.c2`.

In [5]:
%%hive

SELECT * FROM tbl0 INNER JOIN (
         SELECT key as key, value as value
         FROM tbl1
         lateral view explode(c4) dummy_table AS key, value
) tbl_1 ON tbl0.c2 = tbl_1.key;

1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	3
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	5
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	1
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	5
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	5
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	2
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	4
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	3
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	5
1	D	4	2016-06-25	["a","e","c","d"]	{"bb":10,"dd":20,"cc":40}	D	1
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}	C	1
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}	C	0
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}	C	3
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}	C	3
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}	C	2
2	C	4	2015-05-14	["a","c"]	{"dd":40,"bb":20,"cc":10}	C	1
2	C	4	20

---

In [ ]:
!rm tbl* 